In [3]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Install MONAI and Gradio
!pip install -q monai[all] gradio

# Step 3: Imports
import os
import torch
import numpy as np
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
from monai.networks.nets import densenet121
from sklearn.metrics import classification_report, confusion_matrix
import gradio as gr

# Step 4: Paths
data_dir = "/content/drive/My Drive/MONAI_Pneumonai_Classification/dataset"
model_path = "/content/drive/My Drive/MONAI_Pneumonai_Classification/pneumonia_densenet121.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Step 5: Dataset
transform = transforms.Compose([
    transforms.Grayscale(),              # Ensure 1 channel
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = ImageFolder(os.path.join(data_dir, 'train'), transform=transform)
val_dataset = ImageFolder(os.path.join(data_dir, 'val'), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Step 6: Model Setup
model = densenet121(spatial_dims=2, in_channels=1, out_channels=2).to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Step 7: Training
epochs = 5
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(train_loader):.4f}")

# Step 8: Save Model
torch.save(model.state_dict(), model_path)
print(f"✅ Model saved to: {model_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/5, Loss: 0.1794
Epoch 2/5, Loss: 0.1093
Epoch 3/5, Loss: 0.0750
Epoch 4/5, Loss: 0.0626
Epoch 5/5, Loss: 0.0454
✅ Model saved to: /content/drive/My Drive/MONAI_Pneumonai_Classification/pneumonia_densenet121.pth


In [4]:
# Step 9: Evaluation
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

# Results
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", cm)

report = classification_report(y_true, y_pred, target_names=train_dataset.classes)
print("Classification Report:\n", report)


Confusion Matrix:
 [[3 5]
 [0 8]]
Classification Report:
               precision    recall  f1-score   support

      NORMAL       1.00      0.38      0.55         8
   PNEUMONIA       0.62      1.00      0.76         8

    accuracy                           0.69        16
   macro avg       0.81      0.69      0.65        16
weighted avg       0.81      0.69      0.65        16



In [5]:
# Step 10: Gradio UI for Prediction
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

def predict_image(img):
    import PIL
    img = img.convert("L").resize((224, 224))  # grayscale
    tensor = transforms.ToTensor()(img).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(tensor)
        prob = torch.nn.functional.softmax(output[0], dim=0)
        pred_idx = prob.argmax().item()
    return {
        train_dataset.classes[0]: float(prob[0]),
        train_dataset.classes[1]: float(prob[1])
    }

gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=2),
    title="Pneumonia Detection from Chest X-ray"
).launch()


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://13aa98fe4fdb46fda4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
